WB and SVM

In [17]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from imblearn.over_sampling import SMOTE
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, classification_report


df = pd.read_csv('feature_file_1740153842.csv')


wristband_columns = [col for col in df.columns if col.startswith(('bvp', 'gsr', 'tmp'))]
wristband_columns.append('emotion')  # Add emotion column as labels
df_wristband = df[wristband_columns]


label_encoder = LabelEncoder()
df_wristband['emotion'] = label_encoder.fit_transform(df_wristband['emotion'])


X_wristband = df_wristband.drop(columns=['emotion'])
y_wristband = df_wristband['emotion']


smote = SMOTE(random_state=42)
X_resampled_wristband, y_resampled_wristband = smote.fit_resample(X_wristband, y_wristband)


scaler = StandardScaler()
X_resampled_wristband = scaler.fit_transform(X_resampled_wristband)


X_train_wristband, X_test_wristband, y_train_wristband, y_test_wristband = train_test_split(
    X_resampled_wristband, y_resampled_wristband, test_size=0.2, random_state=42
)


svm_wristband = SVC(kernel='rbf', C=10, gamma=0.1, random_state=42)
svm_wristband.fit(X_train_wristband, y_train_wristband)


y_pred_wristband = svm_wristband.predict(X_test_wristband)


accuracy_wristband = np.mean(y_pred_wristband == y_test_wristband)
print(f"Wristband Test Accuracy: {accuracy_wristband:.4f}")


conf_matrix_wristband = confusion_matrix(y_test_wristband, y_pred_wristband)
conf_matrix_df_wristband = pd.DataFrame(
    conf_matrix_wristband,
    index=label_encoder.classes_,
    columns=label_encoder.classes_
)
print("Wristband Confusion Matrix:")
print(conf_matrix_df_wristband)


print("Wristband Classification Report:")
print(classification_report(y_test_wristband, y_pred_wristband, target_names=label_encoder.classes_))

C:\Users\Abdur Rafay\AppData\Local\Temp\ipykernel_11928\3181896823.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_wristband['emotion'] = label_encoder.fit_transform(df_wristband['emotion'])


Wristband Test Accuracy: 0.6239
Wristband Confusion Matrix:
         Excited  Fear  Happy  Sad  relax
Excited       60     3      2    6      2
Fear           8    38      9    8      2
Happy          3     7     38   12      1
Sad           10     6      5   40      8
relax          7     6      2   22     38
Wristband Classification Report:
              precision    recall  f1-score   support

     Excited       0.68      0.82      0.75        73
        Fear       0.63      0.58      0.61        65
       Happy       0.68      0.62      0.65        61
         Sad       0.45      0.58      0.51        69
       relax       0.75      0.51      0.60        75

    accuracy                           0.62       343
   macro avg       0.64      0.62      0.62       343
weighted avg       0.64      0.62      0.62       343



EEG and SVM

In [26]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.utils import class_weight
from imblearn.over_sampling import SMOTE
import random
import matplotlib.pyplot as plt
import seaborn as sns


seed_value = 42
np.random.seed(seed_value)
random.seed(seed_value)


df = pd.read_csv('feature_file_1740153842.csv')


eeg_columns = [col for col in df.columns if not col.startswith(('bvp', 'gsr', 'tmp'))]
df_eeg = df[eeg_columns]


label_encoder = LabelEncoder()
df_eeg['emotion'] = label_encoder.fit_transform(df_eeg['emotion'])


X_eeg = df_eeg.drop(columns=['emotion'])
y_eeg = df_eeg['emotion']


def augment_data(X, y, noise_factor=0.1, num_augmented_per_class=500):
    np.random.seed(seed_value)
    X_augmented = []
    y_augmented = []
    
   
    class_counts = np.bincount(y)

    
    for class_index in range(len(class_counts)):
        num_to_generate = max(0, num_augmented_per_class - class_counts[class_index])
        for _ in range(num_to_generate):
            # Randomly select an index of the original data from the current class
            idx = np.random.choice(np.where(y == class_index)[0])
            noise = noise_factor * np.random.randn(*X[idx].shape)
            X_augmented.append(X[idx] + noise)
            y_augmented.append(y[idx])
    
    return np.array(X_augmented), np.array(y_augmented)


X_augmented, y_augmented = augment_data(X_eeg.values, y_eeg.values)


X_combined = np.vstack((X_eeg.values, X_augmented))
y_combined = np.concatenate((y_eeg.values, y_augmented))


X_train, X_test, y_train, y_test = train_test_split(
    X_combined, y_combined, test_size=0.2, random_state=seed_value, stratify=y_combined
)


scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


class_weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train),
    y=y_train
)


class_weights_dict = {i: class_weights[i] for i in range(len(class_weights))}


svm_model = SVC(kernel='rbf', C=10, class_weight=class_weights_dict, max_iter=10000, random_state=seed_value)


svm_model.fit(X_train, y_train)


y_pred = svm_model.predict(X_test)


train_accuracy = accuracy_score(y_train, svm_model.predict(X_train))
test_accuracy = accuracy_score(y_test, y_pred)


print(f"Training Accuracy: {train_accuracy:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")


print("\nClassification Report:\n", classification_report(y_test, y_pred, target_names=label_encoder.classes_))


conf_matrix = confusion_matrix(y_test, y_pred)
print("\nConfusion Matrix:\n", conf_matrix)


C:\Users\Abdur Rafay\AppData\Local\Temp\ipykernel_11928\1524770464.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_eeg['emotion'] = label_encoder.fit_transform(df_eeg['emotion'])


Training Accuracy: 0.9370
Test Accuracy: 0.7380

Classification Report:
               precision    recall  f1-score   support

     Excited       0.80      0.68      0.74       100
        Fear       0.68      0.82      0.75       100
       Happy       0.82      0.75      0.78       100
         Sad       0.62      0.70      0.66       100
       relax       0.81      0.74      0.77       100

    accuracy                           0.74       500
   macro avg       0.75      0.74      0.74       500
weighted avg       0.75      0.74      0.74       500


Confusion Matrix:
 [[68 20  6  6  0]
 [ 4 82  3  8  3]
 [ 2  2 75 17  4]
 [ 5  8  7 70 10]
 [ 6  8  1 11 74]]


SVM with combined

In [30]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.utils import class_weight
import random
import matplotlib.pyplot as plt
import seaborn as sns


seed_value = 42
np.random.seed(seed_value)
random.seed(seed_value)


df = pd.read_csv('feature_file_1740153842.csv')


eeg_columns = [col for col in df.columns if not col.startswith(('bvp', 'gsr', 'tmp'))]
wristband_columns = [col for col in df.columns if col.startswith(('bvp', 'gsr', 'tmp'))]
combined_columns = eeg_columns + wristband_columns
df_combined = df[combined_columns]


label_encoder = LabelEncoder()
df_combined['emotion'] = label_encoder.fit_transform(df_combined['emotion'])


X_combined = df_combined.drop(columns=['emotion'])
y_combined = df_combined['emotion']


def augment_data(X, y, noise_factor=0.1, num_augmented_per_class=500):
    np.random.seed(seed_value)
    X_augmented = []
    y_augmented = []
    
    
    class_counts = np.bincount(y)

    
    for class_index in range(len(class_counts)):
        num_to_generate = max(0, num_augmented_per_class - class_counts[class_index])
        for _ in range(num_to_generate):
           
            idx = np.random.choice(np.where(y == class_index)[0])
            noise = noise_factor * np.random.randn(*X[idx].shape)
            X_augmented.append(X[idx] + noise)
            y_augmented.append(y[idx])
    
    return np.array(X_augmented), np.array(y_augmented)


X_augmented, y_augmented = augment_data(X_combined.values, y_combined.values)


X_combined_augmented = np.vstack((X_combined.values, X_augmented))
y_combined_augmented = np.concatenate((y_combined.values, y_augmented))


X_train, X_test, y_train, y_test = train_test_split(
    X_combined_augmented, y_combined_augmented, test_size=0.2, random_state=seed_value, stratify=y_combined_augmented
)


scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


class_weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train),
    y=y_train
)


class_weights_dict = {i: class_weights[i] for i in range(len(class_weights))}


svm_model = SVC(kernel='rbf', C=10, class_weight=class_weights_dict, max_iter=10000, random_state=seed_value)


svm_model.fit(X_train, y_train)


y_pred = svm_model.predict(X_test)


train_accuracy = accuracy_score(y_train, svm_model.predict(X_train))
test_accuracy = accuracy_score(y_test, y_pred)


print(f"Training Accuracy: {train_accuracy:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")


print("\nClassification Report:\n", classification_report(y_test, y_pred, target_names=label_encoder.classes_))


conf_matrix = confusion_matrix(y_test, y_pred)
print("\nConfusion Matrix:\n", conf_matrix)



Training Accuracy: 0.9520
Test Accuracy: 0.7480

Classification Report:
               precision    recall  f1-score   support

     Excited       0.73      0.77      0.75       100
        Fear       0.78      0.80      0.79       100
       Happy       0.77      0.72      0.74       100
         Sad       0.67      0.72      0.70       100
       relax       0.79      0.73      0.76       100

    accuracy                           0.75       500
   macro avg       0.75      0.75      0.75       500
weighted avg       0.75      0.75      0.75       500


Confusion Matrix:
 [[77  9 10  1  3]
 [ 5 80  3  8  4]
 [ 7  1 72 17  3]
 [ 7  7  5 72  9]
 [ 9  5  4  9 73]]
